In [15]:
import pandas as pd
import numpy as np
import os
import re
import warnings
import gc

warnings.filterwarnings('ignore')

# ==============================================================================
# FUNGSI PEMBERSIH NAMA UNIT (GLOBAL)
# ==============================================================================
def clean_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    name = name.replace("FORKLIFT", "FORKLIF")
    name = re.sub(r'[^A-Z0-9]', '', name)
    return name

# ==============================================================================
# BLOK 1: LOAD MASTER DATA & MAPPING
# ==============================================================================
file_master = 'cost & bbm 2022 sd 2025 HP.xlsx'
master_data_map = {} 
master_keys_set = set()

if os.path.exists(file_master):
    df_map = pd.read_excel(file_master, sheet_name='Sheet2', header=1)
    
    if 'NAMA ALAT BERAT' in df_map.columns:
        df_map.rename(columns={
            'NAMA ALAT BERAT': 'Unit_Original',
            'ALAT BERAT': 'Jenis_Alat',
            'HP': 'Horse_Power', 
            'CAP': 'Capacity', # [TAMBAHAN]: Membaca kolom Capacity
            'DES 2025': 'Lokasi'
        }, inplace=True)
    
    df_map.dropna(subset=['Unit_Original'], inplace=True)
    df_map['Unit_ID'] = df_map['Unit_Original'].apply(clean_unit_name)
    
    df_map = df_map[~df_map['Unit_Original'].astype(str).str.upper().str.contains('DUMMY', na=False)]
    df_map = df_map[~df_map['Unit_Original'].astype(str).str.upper().str.contains('FALCON', na=False)]
    
    df_map['Horse_Power'] = pd.to_numeric(df_map['Horse_Power'], errors='coerce').fillna(0)
    df_map['Capacity'] = pd.to_numeric(df_map['Capacity'], errors='coerce').fillna(0) # [TAMBAHAN]: Pastikan angka
    
    for _, row in df_map.iterrows():
        clean_id = row['Unit_ID']
        if clean_id:
            master_data_map[clean_id] = {
                'Unit_Name': row['Unit_Original'],
                'Jenis_Alat': row['Jenis_Alat'],
                'Horse_Power': row['Horse_Power'], 
                'Capacity': row['Capacity'], # [TAMBAHAN]: Simpan Capacity
                'Lokasi': row['Lokasi']
            }
            master_keys_set.add(clean_id)
            
    print(f"✅ Mapping Selesai. {len(master_data_map)} unit siap dicocokkan (Exclude FALCON).")

✅ Mapping Selesai. 244 unit siap dicocokkan (Exclude FALCON).


In [16]:
# ==============================================================================
# BLOK 2: BACA EXCEL TRANSAKSI & SUPER SMART MATCHING
# ==============================================================================
file_bbm = 'BBM AAB.xlsx'
target_sheets = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV']
raw_data_list = []
matched_units_count = set()

if os.path.exists(file_bbm) and master_data_map:
    xls = pd.ExcelFile(file_bbm)
    
    for sheet in target_sheets:
        if sheet in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet, header=None)
            
            unit_names_row = df.iloc[0].ffill()
            headers = df.iloc[2]
            dates = df.iloc[3:, 0]
            
            for col in range(1, df.shape[1]):
                header_str = str(headers[col]).strip().upper()
                
                if header_str in ['HM', 'LITER', 'KELUAR', 'PEMAKAIAN']:
                    raw_unit_name = str(unit_names_row[col]).strip().upper()
                    
                    if raw_unit_name == "" or "UNNAMED" in raw_unit_name or "EQUIP NAME" in raw_unit_name or "TOTAL" in raw_unit_name:
                        continue
                    if raw_unit_name.startswith(('GENSET', 'KOMPRESSOR', 'MESIN', 'TANGKI', 'SPBU', 'MOBIL')):
                        continue
                    
                    clean_trx_id = clean_unit_name(raw_unit_name)
                    matched_id = None
                    
                    if "FL RENTAL 01" in raw_unit_name and "TIMIKA" not in raw_unit_name:
                        temp_id = clean_unit_name("FL RENTAL 01 TIMIKA")
                        if temp_id in master_data_map: matched_id = temp_id
                    elif "TOBATI" in raw_unit_name and "KALMAR 32T" in raw_unit_name:
                        temp_id = clean_unit_name("TOP LOADER KALMAR 35T/TOBATI") 
                        if temp_id in master_data_map: matched_id = temp_id
                    elif "L 8477 UUC" in raw_unit_name:
                        temp_id = clean_unit_name("L 9902 UR / S75")
                        if temp_id in master_data_map: matched_id = temp_id
                    
                    if not matched_id and clean_trx_id in master_data_map:
                        matched_id = clean_trx_id
                    if not matched_id and "EX." in raw_unit_name:
                        try:
                            clean_after_ex = clean_unit_name(raw_unit_name.split("EX.")[-1].replace(")", "").strip())
                            if clean_after_ex in master_data_map: matched_id = clean_after_ex
                            elif clean_after_ex:
                                for m_key in master_keys_set:
                                    if clean_after_ex in m_key: matched_id = m_key; break
                        except: pass
                    if not matched_id and " (" in raw_unit_name:
                        try:
                            clean_before = clean_unit_name(raw_unit_name.split(" (")[0].strip())
                            if clean_before in master_data_map: matched_id = clean_before
                        except: pass

                    if matched_id:
                        matched_units_count.add(matched_id)
                        metric_type = 'HM' if header_str == 'HM' else 'LITER'
                        vals = pd.to_numeric(df.iloc[3:, col], errors='coerce')
                        
                        info_unit = master_data_map[matched_id]
                        
                        temp_df = pd.DataFrame({
                            'Date': dates,
                            'Unit_Name': info_unit['Unit_Name'], 
                            'Benchmark_Group': f"{info_unit['Horse_Power']:g} HP",
                            'Lokasi': info_unit['Lokasi'],
                            'Jenis_Alat': info_unit['Jenis_Alat'],
                            'Horse_Power': info_unit['Horse_Power'],
                            'Capacity': info_unit['Capacity'], # [TAMBAHAN]: Masukkan Capacity
                            'Metric': metric_type,
                            'Value': vals
                        })
                        
                        temp_df.dropna(subset=['Value', 'Date'], inplace=True)
                        if not temp_df.empty:
                            raw_data_list.append(temp_df)

if raw_data_list:
    df_all = pd.concat(raw_data_list, ignore_index=True)
    
    df_all['Date'] = pd.to_datetime(df_all['Date'], dayfirst=True, errors='coerce')
    df_all.dropna(subset=['Date'], inplace=True)
    
    # [TAMBAHAN]: Memasukkan 'Capacity' ke dalam index pivot agar tidak hilang
    df_pivot = df_all.pivot_table(
        index=['Unit_Name', 'Benchmark_Group', 'Lokasi', 'Jenis_Alat', 'Horse_Power', 'Capacity', 'Date'],
        columns='Metric',
        values='Value',
        aggfunc='sum'
    ).reset_index()
    
    if 'HM' not in df_pivot.columns: df_pivot['HM'] = 0
    if 'LITER' not in df_pivot.columns: df_pivot['LITER'] = 0
    df_pivot['HM'] = df_pivot['HM'].fillna(0)
    df_pivot['LITER'] = df_pivot['LITER'].fillna(0)
    
    df_pivot.sort_values(by=['Unit_Name', 'Date'], inplace=True)
    
    df_pivot['HM_Clean'] = df_pivot['HM'].replace(0, np.nan)
    df_pivot['HM_Clean'] = df_pivot.groupby('Unit_Name')['HM_Clean'].ffill()
    df_pivot['HM_Clean'] = df_pivot['HM_Clean'].fillna(0)
    
    df_pivot['Delta_HM'] = df_pivot.groupby('Unit_Name')['HM_Clean'].diff().fillna(0)
    
    df_pivot.loc[df_pivot['Delta_HM'] < 0, 'Delta_HM'] = 0 
    df_pivot.loc[df_pivot['Delta_HM'] > 100, 'Delta_HM'] = 0 
    
    print(f"✅ Data Transaksi Selesai. Hitungan HM dijamin kembali akurat.")

✅ Data Transaksi Selesai. Hitungan HM dijamin kembali akurat.


In [17]:
# ==============================================================================
# BLOK 3: HITUNG RATIO, BENCHMARK HP, & EXPORT EXCEL (UPDATE CAPACITY)
# ==============================================================================
if 'df_pivot' in locals() and not df_pivot.empty:
    
    # [TAMBAHAN]: Memasukkan 'Capacity' ke dalam groupby akhir
    final_stats = df_pivot.groupby(['Unit_Name', 'Benchmark_Group', 'Lokasi', 'Jenis_Alat', 'Horse_Power', 'Capacity']).agg({
        'LITER': 'sum',
        'Delta_HM': 'sum'
    }).reset_index()
    
    final_stats.rename(columns={'LITER': 'Total_Liter', 'Delta_HM': 'Total_HM_Work'}, inplace=True)
    
    final_stats['Fuel_Ratio'] = final_stats.apply(lambda row: row['Total_Liter'] / row['Total_HM_Work'] if row['Total_HM_Work'] > 0 else 0, axis=1)
    
    df_valid = final_stats[(final_stats['Total_HM_Work'] > 0) & (final_stats['Total_Liter'] > 0)].copy()
    benchmark_stats = df_valid.groupby('Benchmark_Group')['Fuel_Ratio'].median().reset_index()
    benchmark_stats.rename(columns={'Fuel_Ratio': 'Group_Benchmark_Median'}, inplace=True)
    
    df_final = pd.merge(final_stats, benchmark_stats, on='Benchmark_Group', how='left')
    
    def get_status(row):
        if row['Total_HM_Work'] <= 0 or row['Total_Liter'] <= 0: 
            return "INAKTIF"
        bench = row['Group_Benchmark_Median']
        return "EFISIEN" if row['Fuel_Ratio'] <= bench else "BOROS"

    df_final['Performance_Status'] = df_final.apply(get_status, axis=1)
    
    df_final['Potensi_Pemborosan_Liter'] = 0
    boros_mask = df_final['Performance_Status'] == "BOROS"
    df_final.loc[boros_mask, 'Potensi_Pemborosan_Liter'] = (
        (df_final.loc[boros_mask, 'Fuel_Ratio'] - df_final.loc[boros_mask, 'Group_Benchmark_Median']) 
        * df_final.loc[boros_mask, 'Total_HM_Work']
    )
    
    # Pembulatan 2 Angka Di Belakang Koma
    df_final['Fuel_Ratio'] = df_final['Fuel_Ratio'].round(2)
    df_final['Group_Benchmark_Median'] = df_final['Group_Benchmark_Median'].round(2)
    df_final['Potensi_Pemborosan_Liter'] = df_final['Potensi_Pemborosan_Liter'].round(2)
    
    df_active = df_final[df_final['Performance_Status'] != "INAKTIF"].copy()
    df_inactive = df_final[df_final['Performance_Status'] == "INAKTIF"].copy()
    
    output_file = 'Benchmark_Per_HP_Alat_Berat.xlsx'
    try:
        with pd.ExcelWriter(output_file) as writer:
            # [TAMBAHAN]: Menampilkan 'Capacity' di Output Excel Aktif
            cols = ['Benchmark_Group', 'Jenis_Alat', 'Capacity', 'Horse_Power', 'Lokasi', 'Unit_Name', 
                    'Total_Liter', 'Total_HM_Work', 'Fuel_Ratio', 
                    'Group_Benchmark_Median', 'Performance_Status', 'Potensi_Pemborosan_Liter']
            df_active[cols].to_excel(writer, sheet_name='Unit_Aktif', index=False)
            
            summary_grp = df_active.groupby('Benchmark_Group').agg({
                'Unit_Name': 'count',
                'Total_Liter': 'sum',
                'Total_HM_Work': 'sum',
                'Group_Benchmark_Median': 'first',
                'Potensi_Pemborosan_Liter': 'sum'
            }).reset_index().rename(columns={'Unit_Name': 'Populasi_Unit'})
            summary_grp.to_excel(writer, sheet_name='Summary_Group', index=False)
            
            # [TAMBAHAN]: Menampilkan 'Capacity' di Output Excel Inaktif
            df_inactive[['Benchmark_Group', 'Jenis_Alat', 'Capacity', 'Horse_Power', 'Unit_Name', 'Total_Liter', 'Total_HM_Work']].to_excel(writer, sheet_name='Unit_Inaktif', index=False)
            
        print(f"\n💾  SUKSES! Laporan Tersimpan: '{output_file}'")

    except Exception as e:
        print(f"⚠️ Gagal save Excel: {e}")


💾  SUKSES! Laporan Tersimpan: 'Benchmark_Per_HP_Alat_Berat.xlsx'
